In [1]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
from itertools import groupby
import datetime as dt

#visualizing results
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#import yellowbrick as yb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

## Task info  

- Event stamps ( array I )
    - TOs and ITIs:
        - 7   = Time-out, no lever response choice in 10 sec
        - 19  = Time-out, no right lever response forced in 10 seconds
        - 20  = Time-out, no left lever response forced in 10 seconds
        - 16  = ITI over

    - LPs and reinforcement:
        - 17  = Initial forced trial lever press right
        - 18  = Initial forced trial lever press left
        - 8   = Lever choice right
        - 9   = Lever choice left
        - 5   = Reinforcement delivery
    - HEs:
        - 10  = Initial head entry in trial
            - **** IN VERSION 22_BERNSTEIN THIS IS SIMULATED AFTER 1 SECOND ** VERSION 23 REINSTATED
        - 11  = Subsequent head entries
        
Don't Use

-   1   = Rt lever response in FR (if FR greater than 1 otherwise see codes 8,9,17,18,21,22)
-   2   = Lt lever response (if FR greater than 1 otherwise see codes 8,9,17,18,21,22)
-   3   = Rewarded rt lever response
-   4   = Rewarded lt lever response
-   6   = Time-out, no initial head-entry in 10 sec   **** REMOVED FOR VERSION 22_BERNSTEIN ** REINSTATED VERSION 23
-   12  = Right lever press during time-out
-   13  = Left lever press during time-out
-   14  = Head entry during time-out
-   15  = Time-out over
-   21  = Forced trial right lever
-   22  = Forced trail left lever
-   23  = Choice trial
-   24  = Head entry during ITI
-   25  = ICS signal
-   26  = Extinction trial
-   27  = Time-out, no press in A(22) minutes
-   28  = Time-out, did not complete FR in A(21) minutes
-   100 = Session termination




## Variables check

Use these to check that correct parameters were entered into the Med-PC program for each file

- A(4) = Reward on right lever
- A(5) = Reward on left lever
- A(24) = Probabilistic lever (0 = None, 1 = Right, 2 = Left, 3 = Both)
- A(10) = Forced or choice (1 = forced, 2 = choice)


Don't Use

- A(1) = Intertrial interval calculated 
- A(2) = FR for right lever
- A(3) = FR for left lever
- A(6) = Delay on right lever
- A(7) = Delay on left lever
- A(8) = Lever choice (1 = right, -1 = left lever)
- A(9) = Maximum session length
- A(11) = Time to complete FR in seconds 
- A(12) = Time-out length
- A(13) = Maximum FR
- A(14) = Maximum intertrial interval (60 seconds default)
- A(15) = Minimum intertrial interval (30 seconds default)
- A(16) = Counter for trial number
- A(17) = Maximum number of trials (48 default)
- A(18) = Time limit defining a miss (10 seconds default)
- A(19) = Counter for completed right lever FRs
- A(20) = Counter for completed left lever FRs
- A(21) = Time-limit for completing an FR (minutes)
- A(22) = Time-limit for between presses; i.e., if a press is not given at least every A(22) minutes, a miss occurs
- A(23) = Repeat missed trials? (1:YES   2:NO)
- A(25) = Chosen from P list of reinforcement or no reinforcement
- A(26) = Counter for reinf probs list

## Functions

In [21]:
def get_file_info(file_path_name):
    # takes in a file path, finds the date, animal_number, session, session number, and sub_session, saves each accordingly
    import re
    
    task = file_path_name.split('/')[-1].split('_')[0]
    probability = file_path_name.split('/')[-1].split('_')[1].split('.')[0]
    sub_session = int(file_path_name.split('/')[-1].split('_')[1].split('.')[1])
    animal = int(file_path_name.split('/')[-1].split('_')[2])
    date = file_path_name.split('/')[-1].split('_')[3]

    return task, probability, sub_session, animal, date

In [22]:
def make_array_from_path (file_path_name):
    # takes in file path, creates array (each row of array is a row from original med file)
    import numpy as np
    import pandas as pd
    
    data = pd.read_table(file_path_name)
    data_table = pd.DataFrame(data = data)
    data_array = data_table.values
    
    return data_array

In [23]:
def get_events_and_times(array, event_letter, time_letter):
    #takes in array and the letters Med PC program uses to signify an event_stamp and time_stamp; 
    #finds all event_stamps and corresponding time_stamps; creats pandas dataframe
    import re
    import numpy as np
    import pandas as pd

    i = 0
    event_data = []
    time_data = []
    while i < len(array):
        if re.search(str(event_letter + ':'), str(array[i])):
            i = i + 1
            while re.search('\[\'\ ', str(array[i])):
                split = array[i][0].split()[1:]
                for element in split:
                    event_data.append(float(element))
                i = i + 1
        if re.search(str(time_letter + ':'), str(array[i])):
            i = i + 1
            while re.search('\[\'\ ', str(array[i])):
                split = array[i][0].split()[1:]
                for element in split:
                    time_data.append(float(element))
                i = i + 1
        i = i + 1
    
    eandt = np.column_stack((event_data, time_data))
    es_et = pd.DataFrame(data = eandt)
    es_et.columns = ['event_stamp', 'time_stamp']
    
    return es_et

In [24]:
def get_trial_times_two(events_and_times, start_marker, ITI_marker, TO_rt_marker, TO_lt_marker, TO_choice_marker):
    #takes in dataframe of events and corresponding time_stamps and the Med PC program code for trial related markers
    #e.g. conditioned stimulus onset and offset, ITI onset and offset
    #finds time_stamps for each start and end, puts into new dataframe
    import numpy as np
    import pandas as pd

    trial_starts = events_and_times[events_and_times['event_stamp'] == start_marker]['time_stamp'].values
    trial_ends = events_and_times[events_and_times['event_stamp'] == ITI_marker]['time_stamp'].values
    trial_lp_TOs = events_and_times[(events_and_times['event_stamp'] == TO_rt_marker) | (events_and_times['event_stamp'] == TO_lt_marker) | (events_and_times['event_stamp'] == TO_choice_marker)]['time_stamp'].values
    
    trial_ends_updated = np.sort(np.concatenate((trial_ends, trial_lp_TOs), axis=0))
    
    times = pd.DataFrame(data = [trial_starts, trial_ends_updated])
    times = times.T
    times.columns = ['trial_start', 'trial_end']
    
    return times

In [25]:
def create_trials_df(events_and_times, trial_times):
    #takes in df of events and times and df of trial times
    #creates tidy df containing events and times according to trials
    i = 0
    trial_data = pd.DataFrame()
    
    while i < len(trial_times):
        data = events_and_times[(events_and_times['time_stamp'] >= trial_times['trial_start'][i]) & (events_and_times['time_stamp'] <= trial_times['trial_end'][i])]
        data['trial_#'] = [i]*len(data)
        data['trial_start'] = trial_times['trial_start'][i]
        data['trial_end'] = trial_times['trial_end'][i]
        trial_data = pd.concat([trial_data, data], axis = 0, ignore_index=True)
        i = i + 1
    
    return trial_data

In [26]:
def count(trial_data, event_num):
    #get count of event_num
    trials = len((trial_data['trial_#'].unique()))
    i = 0
    count = []
    while i < trials:
        data = trial_data[trial_data['trial_#'] == i]
        count.append(len(data[data['event_stamp'] == event_num]))
        i = i + 1
    
    return count

In [27]:
def latency(trial_data, event_num):
    #get latency of time between trial_start and event_num
    trials = len((trial_data['trial_#'].unique()))
    i = 0
    latency_1st = []
    latency_ave = []
    while i < trials:
        data = trial_data[trial_data['trial_#'] == i]
        data = data[data['event_stamp'] == event_num]
        if len(data) < 1:
            latency_1st.append(0.0)
            latency_ave.append(0.0)
        else:
            latency_1st.append((data['time_stamp'] - data['trial_start']).values[0]/100)
            latency_ave.append((data['time_stamp'] - data['trial_start']).values.mean()/100)
        i = i + 1
    
    return latency_1st, latency_ave

In [28]:
def create_session_df(trial_data, event_dic):
    
    trials = len((trial_data['trial_#'].unique()))
    
    #get HE_wait times
    times_df = pd.DataFrame(data = [trial_data.groupby('trial_#')['trial_start'].unique(), trial_data.groupby('trial_#')['trial_end'].unique()])
    times_df = times_df.T
    
    i = 0
    HE_waits = []
    while i < times_df.shape[0]:
        if (i == 0) | (i == times_df.shape[0] - 1):
            HE_waits.append(np.nan)
            i += 1
        else:
            HE_wait = times_df.iloc[i]['trial_start'] - times_df.iloc[i-1]['trial_end']
            HE_waits.append(int(HE_wait/100))
            i += 1
    
    session_data = pd.DataFrame(index=np.arange(0,trials))

    for name, event in event_dic.items():
        session_data[str(name + '_count')] = count(trial_data, event)
        session_data[str(name + '_latency_1st')], session_data[str(name + '_latency_ave_bit')] = latency(trial_data, event)
    
    session_data['trial_num'] = np.arange(session_data.shape[0])
    session_data['HE_wait'] = HE_waits
    
    return session_data

In [29]:
def get_variables(array, variable_letter):
    #takes in array and the letter Med PC program uses to encode variables (parameters for behavioral program entered at time of exp)
    import re
    import numpy as np
    import pandas as pd

    i = 0
    variable_data = []
    while i < len(array):
        if re.search(str(variable_letter + ':'), str(array[i])):
            i = i + 1
            while re.search('\[\'\ ', str(array[i])):
                split = array[i][0].split()[1:]
                for element in split:
                    variable_data.append(float(element))
                i = i + 1
        i = i + 1
    
    variable_df = pd.DataFrame(data = variable_data)
    variable_df.columns = ['variables']
    
    return variable_df

In [43]:
def check_parameter_entry(session_data, animal_info):
    
    #list of session names that we want to check (eg any session that used a probability)
    probabilities = ['75', '50', '25']
    
    animal = int(session_data['animal'].unique())
    
    #for 100 probability
    if session_data['probability'].unique() == '100':
        #determine high reward lever assigned to animal and fill in corresponding session data
        if animal_info[animal_info['animal'] == animal]['lever'].values == 1:
            session_data['rt_reward'] = 2
            session_data['lt_reward'] = 1
            session_data['prob_lever'] = 0
        if animal_info[animal_info['animal'] == animal]['lever'].values == 2:
            session_data['rt_reward'] = 1
            session_data['lt_reward'] = 2
            session_data['prob_lever'] = 0
            
    #for probabilities 75, 50, 25
    elif session_data['probability'].unique() in probabilities:
        #determine high reward lever assigned to animal and fill in corresponding session data
        if animal_info[animal_info['animal'] == animal]['lever'].values == 1:
            session_data['rt_reward'] = 2
            session_data['lt_reward'] = 1
            session_data['prob_lever'] = 1
        if animal_info[animal_info['animal'] == animal]['lever'].values == 2:
            session_data['rt_reward'] = 1
            session_data['lt_reward'] = 2
            session_data['prob_lever'] = 2
            
    #for 0 probability trial
    elif session_data['probability'].unique() == '0':
        #determine high reward lever assigned to animal and fill in corresponding session data
        if animal_info[animal_info['animal'] == animal]['lever'].values == 1:
            session_data['rt_reward'] = 0
            session_data['lt_reward'] = 1
            session_data['prob_lever'] = 0
        if animal_info[animal_info['animal'] == animal]['lever'].values == 2:
            session_data['rt_reward'] = 1
            session_data['lt_reward'] = 0
            session_data['prob_lever'] = 0
            
    else:
        return print('This is not a probability session.') 
            
    #assume entries were false until check determines otherwise
    rt_reward = False
    lt_reward = False
    force_choice = False
    prob_lever = False
    
    #get variables data containing parameter entries
    variables_df = get_variables(array, "A")

    #use varibales data and session data to determine if the correct parameters were entered
    if float(variables_df.iloc[4].values) in session_data['rt_reward'].unique():
        rt_reward = True

    if float(variables_df.iloc[5].values) in session_data['lt_reward'].unique():
        lt_reward = True
    
    if float(variables_df.iloc[24].values) in session_data['prob_lever'].unique():
        prob_lever = True
        
    if session_data['sub_session'].unique() == 1:
        sub_session = 1
    if session_data['sub_session'].unique() == 2:
        sub_session = 2 
    if float(variables_df.iloc[10].values) == sub_session:
        force_choice = True
    
    #print results
    return print('rt_reward:', rt_reward, '\n', 'lt_reward:', lt_reward, '\n', 'force_choice:', force_choice, '\n', 'prob_lever:', prob_lever, '\n')
    

## Read in meta data

In [79]:
#read animal info (lever assignment, group etc) into df

path_animal_info = '/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/assignments/PCA_PD_DRL_210101_meta.xlsx'
    
animal_info = pd.read_excel(path_animal_info)
animal_info = pd.DataFrame(data = animal_info)

animal_info.head()

,cage,animal,group,lever
0,279,1116,4,1
1,279,1117,4,1
2,279,1118,4,1
3,283,1128,5,1
4,283,1130,5,1


## Read in task data

In [80]:
path_1 = '/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task'

session_list_1 = os.listdir(path_1)
print(len(session_list_1))
session_list_1[0:5]

141


['PD_0.2_1123_210113',
 'PD_50.1_1130_210111',
 'PD_25.2_1130_210112',
 'PD_75.1_1138_210108',
 'PD_100.2_1123_210107']

In [81]:
file_paths = []
for file in session_list_1:
    file_path = path_1 + '/' + file
    file_paths.append(file_path)

print(len(file_paths))
file_paths[0:5]

141


['/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_0.2_1123_210113',
 '/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_50.1_1130_210111',
 '/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_25.2_1130_210112',
 '/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_75.1_1138_210108',
 '/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_100.2_1123_210107']

## Process

In [85]:
%%time
final_data = pd.DataFrame()

#these are the event_stamps we want analyzed
event_dic = {'reinforcement': 5, 'right_lever_choice': 8, 'left_lever_choice': 9, 'right_lever_force': 17, \
             'left_lever_force': 18, 'HE_trial': 11}

run_variables = False

for file in file_paths:
    print(file)
    
    if file.split('/')[-1] == '.DS_Store':
        continue
    
    #proccess file
    task, probability, sub_session, animal, date = get_file_info(file)
    array = make_array_from_path(file)
    events_and_times = get_events_and_times(array, "I", "T")
    trial_times = get_trial_times_two(events_and_times, 10, 16, 19, 20, 7)
    trial_data = create_trials_df(events_and_times, trial_times)
    session_data = create_session_df(trial_data, event_dic)
    
    #create pandas tidy df
    session_data['task'] = [task]*(session_data.shape[0])
    session_data['date'] = [date]*(session_data.shape[0])
    session_data['animal'] = [animal]*(session_data.shape[0])
    session_data['probability'] = [probability]*(session_data.shape[0])
    session_data['sub_session'] = [sub_session]*(session_data.shape[0])
    
    try:
        session_data['high_reward_lever'] = np.repeat(animal_info[animal_info['animal'] == animal]['lever'].unique(), session_data.shape[0])
    except:
        session_data['high_reward_lever'] = np.nan
    
    #run varibale check
    if run_variables == True:
        check_parameter_entry(session_data, animal_info)
    
    if run_variables == False:
        session_data['rt_reward'] = np.nan
        session_data['lt_reward'] = np.nan
        session_data['prob_lever'] = np.nan
    
    #combine lever force and choice data (use sub_session column to groupby instead)
    if sub_session == 1:
        session_data.rename({'right_lever_force_count': 'right_lever_count',
                          'left_lever_force_count': 'left_lever_count',
                          'right_lever_force_latency_1st': 'right_lever_latency',
                          'left_lever_force_latency_1st': 'left_lever_latency',}, axis='columns', inplace=True)
    elif sub_session == 2:  
        session_data.rename({'right_lever_choice_count': 'right_lever_count',
                          'left_lever_choice_count': 'left_lever_count',
                          'right_lever_choice_latency_1st': 'right_lever_latency',
                          'left_lever_choice_latency_1st': 'left_lever_latency',}, axis='columns', inplace=True)
    else:
        pass

    final_data = pd.concat([final_data, session_data], axis = 0, ignore_index=True)

final_data = final_data[['date', 'animal', 'task', 'probability', 'sub_session', 'trial_num', 'high_reward_lever', 
                         'reinforcement_count', 'right_lever_count', 'right_lever_latency', 
                         'left_lever_count', 'left_lever_latency', 
                         'HE_trial_count', 'HE_trial_latency_1st', 'HE_wait', 'rt_reward', 'lt_reward', 'prob_lever']]

/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_0.2_1123_210113
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_50.1_1130_210111
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_25.2_1130_210112
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_75.1_1138_210108
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_100.2_1123_210107
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_50.1_1119_210114
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_50.2_1138_210111
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_

/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_25.1_1130_210112
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_50.2_1130_210111
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_0.1_1123_210113
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_75.2_1123_210108
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_100.1_1138_210107
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_50.1_1118_210112
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_50.2_1144_210111
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_

/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_0.1_1118_210114
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_0.1_1130_200113
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_75.2_1144_210108
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_75.2_1119_200113
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_50.1_1135_210111
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_25.2_1135_210112
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_100.1_1118_210108
/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_2

In [86]:
final_data.to_csv('PD_task.csv')

In [70]:
run_variables = False

file_path = '/Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_0.2_1124_210113'
#file_path = file_paths[0]
print('file_path:', '\n', file_path, '\n')

task, probability, sub_session, animal, date = get_file_info(file_path)
print('date:', date, 'animal:', animal, 'task:', task, 'probability:', probability, 'sub_session:', sub_session, '\n')

array = make_array_from_path(file_path)
print(array, '\n')

events_and_times = get_events_and_times(array, "I", "T")
print(events_and_times.head(), '\n')

trial_times = get_trial_times_two(events_and_times, 10, 16, 19, 20, 7)
print(trial_times.head(), '\n')

trial_data = create_trials_df(events_and_times, trial_times)
print(trial_data.head(), '\n')

bit_dic = {'reinforcement': 5, 'right_lever_choice': 8, 'left_lever_choice': 9, 'right_lever_force': 17, 'left_lever_force': 18, 'HE_trial': 11}
session_data = create_session_df(trial_data, bit_dic)

session_data['date'] = [date]*(session_data.shape[0])
session_data['animal'] = [animal]*(session_data.shape[0])
session_data['task'] = [task]*(session_data.shape[0])
session_data['probability'] = [probability]*(session_data.shape[0])
session_data['sub_session'] = [sub_session]*(session_data.shape[0])

if run_variables == True:
    check_parameter_entry(session_data, animal_info)
    
if run_variables == False:
    session_data['rt_reward'] = np.nan
    session_data['lt_reward'] = np.nan
    session_data['prob_lever'] = np.nan
    
try:
    session_data['high_reward_lever'] = np.repeat(animal_info[animal_info['animal'] == animal]['lever'].unique(), session_data.shape[0])
except:
    session_data['high_reward_lever'] = np.nan

final_data = pd.DataFrame()

final_data = pd.concat([final_data, session_data], axis = 0, ignore_index=True)

final_data = final_data[['date', 'animal', 'task', 'probability', 'sub_session', 'trial_num', 'high_reward_lever', 
                         'reinforcement_count', 'right_lever_force_count', 'right_lever_force_latency_1st', 
                         'left_lever_force_count', 'left_lever_force_latency_1st', 'right_lever_choice_count', 
                         'right_lever_choice_latency_1st', 'left_lever_choice_count', 'left_lever_choice_latency_1st', 
                         'HE_trial_count', 'HE_trial_latency_1st', 'HE_wait', 'rt_reward', 'lt_reward', 'prob_lever']]

final_data.head()

file_path: 
 /Users/abbieschindler/Documents/Schindler_Lab/Data/Behavior/med_boxes/probability_discounting/r2_210101/task/PD_0.2_1124_210113 

date: 210113 animal: 1124 task: PD probability: 0 sub_session: 2 

[['Start Date: 01/13/21']
 ['End Date: 01/13/21']
 ['Subject: 1124']
 ['Experiment: 0']
 ['Group: 0']
 ['Box: 3']
 ['Start Time: 13:17:38']
 ['End Time: 13:44:08']
 ['MSN: Mark_forced_choice23_Bernstein_50_50_TTL']
 ['G:     386.000']
 ['H:  138221.000']
 ['K:       0.000']
 ['N:       0.000']
 ['O:       0.000']
 ['P:       0.000']
 ['Y:       0.000']
 ['Z:       0.000']
 ['A:']
 ['     0:        0.000       52.000        1.000        1.000        1.000']
 ['     5:        0.000        0.000        0.000        1.000      120.000']
 ['    10:        2.000        0.000        0.000        0.000       45.000']
 ['    15:       45.000       25.000       24.000       10.000       14.000']
 ['    20:       10.000        5.000        1.000        2.000        0.000']
 ['    25:       

,date,animal,task,probability,sub_session,trial_num,high_reward_lever,reinforcement_count,right_lever_force_count,right_lever_force_latency_1st,left_lever_force_count,left_lever_force_latency_1st,right_lever_choice_count,right_lever_choice_latency_1st,left_lever_choice_count,left_lever_choice_latency_1st,HE_trial_count,HE_trial_latency_1st,HE_wait,rt_reward,lt_reward,prob_lever
0,210113,1124,PD,0,2,0,2,1,0,0.0,0,0.0,1,2.03,0,0.00,3,0.13,NaN,NaN,NaN,NaN
1,210113,1124,PD,0,2,1,2,0,0,0.0,0,0.0,0,0.00,1,1.61,3,0.03,0.0,NaN,NaN,NaN
2,210113,1124,PD,0,2,2,2,1,0,0.0,0,0.0,1,0.74,0,0.00,0,0.00,6.0,NaN,NaN,NaN
3,210113,1124,PD,0,2,3,2,0,0,0.0,0,0.0,0,0.00,1,1.60,0,0.00,2.0,NaN,NaN,NaN
4,210113,1124,PD,0,2,4,2,0,0,0.0,0,0.0,0,0.00,1,1.39,1,0.09,0.0,NaN,NaN,NaN
